In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

In [2]:
%load_ext autoreload
%autoreload 1
%aimport analyze

In [3]:
summit = analyze.Analysis('results/summit')

KeyError: 'input'

In [ ]:
print(summit)

In [ ]:
summed = analyze.summarize_instances(summit.result[[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time']])

In [ ]:
occupancy = summit.results['num_steps'] / (summit.results['num_step_iters'] * summit.input['max_num_tracks'])

In [ ]:
summed

In [ ]:
def get_cpugpu_ratio(summary):
    mean = summary['mean'].unstack()
    rel  = (summary['std'] / summary['mean']).unstack()
    ratio = mean['cpu'] / mean['gpu']
    rel_err = np.sqrt(rel['cpu']**2 + rel['gpu']**2)
    return pd.DataFrame({'ratio': ratio, 're': rel_err})

In [ ]:
get_cpugpu_ratio(summed['avg_time_per_step'])

In [ ]:
summit.results.index

In [ ]:
summed[['num_steps', 'unconverged']].xs('mean', axis=1, level='summary')

In [ ]:
def plot_counts(ax, out):
    blue = (.1, .1, .9)
    red = (.7, .1, .1)

    lines = []
    def plot(ax, *args, **kwargs):
        line, = ax.plot(*args, **kwargs)
        lines.append(line)

    plot(ax, out['result']['active'], '-', color=(blue + (0.5,)), label='Active')
    plot(ax, out['result']['alive'], '-', color=blue, label='Alive')
    ax.set_xlabel('Step iteration')
    ax.set_ylabel('Tracks', color=blue)

    oax = ax.twinx()
    inits = np.array(out['result']['initializers'])
    plot(oax, inits, '--', color=red, label='Queued')
    oax.axhline(out['input']['initializer_capacity'], linestyle='--', color=(red + (0.25,)))
    oax.set_ylabel('Initializers', color=red)

    max_init_idx = np.argmax(inits)
    max_init_val = inits[max_init_idx]
    text = re.sub(r'([-+.0-9]+)e\+?(-)?0*([0-9]+)', r'$\1\\times 10^{\2\3}$',
                  f'{max_init_val:.2g}')
    oax.annotate(text + ' queued', xy=(max_init_idx, max_init_val), xycoords='data',
                 xytext=(30, 10), textcoords='offset points',
                 size='x-small',
                 bbox=dict(boxstyle="round,pad=.2", fc=(0.9, 0.9, 0.9, 0.8) , ec=(0.2,)*3),
                 arrowprops=dict(arrowstyle="->", ec=red, lw=1,
                                 connectionstyle="arc3,rad=0.2"))

    oax.spines['left'].set_color(blue)
    oax.spines['right'].set_color(red)

    ax.legend(lines, [l.get_label() for l in lines])

    return {
        'ax': ax,
        'oax': oax,
    }

In [ ]:
with open('results/summit/cms2018+field+msc-vecgeom-gpu/0.json') as f:
    full = json.load(f)

In [ ]:
(fig, ax) = plt.subplots()
plot_counts(ax, full);